<a href="https://colab.research.google.com/github/hossamfadeel/ML_DL_AI/blob/main/Introduction_to_parallel_programming_pt_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Collaboratory Introduction
Collaboratory is essentially jupyter notebook hosted by Google cloud machines. It comes with everyday python packages already installed, like numpy.

In [ ]:
import numpy as np

As it's google product, you can expect tensorflow :)

In [ ]:
import tensorflow as tf

## Enabling GPU
What's really amazing about collaboratory (or Google's generousity) is that there's also GPU option available.

Follow on the collaboratory menu tabs, "Runtime" => "Change runtime type".

<img src="http://deeplearnphysics.org/Blog/imgs/2018-03-02-Collab-00-img00.png" width="50%">

### Choosing Runtime type
Then you should see a pop-up where you can choose GPU.

<img src="http://deeplearnphysics.org/Blog/imgs/2018-03-02-Collab-00-img01.png" width="30%"> run

After you change your runtime, your runtime should automatically restart (which means information from executed cells disappear). 
### nvidia-smi
If you own GPU you may be familiar with `nvidia-smi`, NVIDIA binary to print out gpu's utilization summary.

Violla! I got T4 or K80 GPU w/ 12GB memory (thanks Google!).  

Now if you want to acquire values in this summary text, youl probably want something else like `gputi`.

Next section demonstrates how.

## Checking GPU devices

### CUDA devices

In [ ]:
!pip -q install gputil psutil humanize
# Import packages
import os,sys,humanize,psutil,GPUtil
 
# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'
    .format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
# Execute function
mem_report()
!nvidia-smi

CPU RAM Free: 12.7 GB
GPU 0 ... Mem Free: 16280MB / 16280MB | Utilization   0%
Wed Oct 21 01:27:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
         

### OpenCL devices

In [ ]:
%%sh 
cat > CL-devices.c << EOF
#include <stdio.h>                                                                                                                                               
#include <stdlib.h>
#include <CL/cl.h>

int main() {

    int i, j;
    char* value;
    size_t valueSize;
    cl_uint platformCount;
    cl_platform_id* platforms;
    cl_uint deviceCount;
    cl_device_id* devices;
    cl_uint maxComputeUnits;

    // get all platforms
    clGetPlatformIDs(0, NULL, &platformCount);
    platforms = (cl_platform_id*) malloc(sizeof(cl_platform_id) * platformCount);
    clGetPlatformIDs(platformCount, platforms, NULL);

    for (i = 0; i < platformCount; i++) {

        // get all devices
        clGetDeviceIDs(platforms[i], CL_DEVICE_TYPE_ALL, 0, NULL, &deviceCount);
        devices = (cl_device_id*) malloc(sizeof(cl_device_id) * deviceCount);
        clGetDeviceIDs(platforms[i], CL_DEVICE_TYPE_ALL, deviceCount, devices, NULL);

        // for each device print critical attributes
        for (j = 0; j < deviceCount; j++) {

            // print device name
            clGetDeviceInfo(devices[j], CL_DEVICE_NAME, 0, NULL, &valueSize);
            value = (char*) malloc(valueSize);
            clGetDeviceInfo(devices[j], CL_DEVICE_NAME, valueSize, value, NULL);
            printf("%d. Device: %s\n", j+1, value);
            free(value);

            // print hardware device version
            clGetDeviceInfo(devices[j], CL_DEVICE_VERSION, 0, NULL, &valueSize);
            value = (char*) malloc(valueSize);
            clGetDeviceInfo(devices[j], CL_DEVICE_VERSION, valueSize, value, NULL);
            printf(" %d.%d Hardware version: %s\n", j+1, 1, value);
            free(value);

            // print software driver version
            clGetDeviceInfo(devices[j], CL_DRIVER_VERSION, 0, NULL, &valueSize);
            value = (char*) malloc(valueSize);
            clGetDeviceInfo(devices[j], CL_DRIVER_VERSION, valueSize, value, NULL);
            printf(" %d.%d Software version: %s\n", j+1, 2, value);
            free(value);

            // print c version supported by compiler for device
            clGetDeviceInfo(devices[j], CL_DEVICE_OPENCL_C_VERSION, 0, NULL, &valueSize);
            value = (char*) malloc(valueSize);
            clGetDeviceInfo(devices[j], CL_DEVICE_OPENCL_C_VERSION, valueSize, value, NULL);
            printf(" %d.%d OpenCL C version: %s\n", j+1, 3, value);
            free(value);

            // print parallel compute units
            clGetDeviceInfo(devices[j], CL_DEVICE_MAX_COMPUTE_UNITS,
                    sizeof(maxComputeUnits), &maxComputeUnits, NULL);
            printf(" %d.%d Parallel compute units: %d\n", j+1, 4, maxComputeUnits);

        }

        free(devices);

    }

    free(platforms);
    return 0;

} 
EOF

In [ ]:
!nvcc -o CL-devices CL-devices.c -lOpenCL && ./CL-devices

1. Device: Tesla P100-PCIE-16GB
 1.1 Hardware version: OpenCL 1.2 CUDA
 1.2 Software version: 418.67
 1.3 OpenCL C version: OpenCL C 1.2 
 1.4 Parallel compute units: 56


## Example 1: Hello world

### C Hello World example

In [ ]:
%%sh 
cat > hello.c << EOF
#include <stdio.h>
 
#define N 16
 
int main(int argc,char **argv)
{
    for(int i = 0; i < N; ++i)
    {
      printf("Hello world! I'm iteration %d\n", i);
    }
 
    printf("That's all!\n");
 
    return 0;
}
EOF

In [ ]:
!gcc -o hello hello.c && ./hello

Hello world! I'm iteration 0
Hello world! I'm iteration 1
Hello world! I'm iteration 2
Hello world! I'm iteration 3
Hello world! I'm iteration 4
Hello world! I'm iteration 5
Hello world! I'm iteration 6
Hello world! I'm iteration 7
Hello world! I'm iteration 8
Hello world! I'm iteration 9
Hello world! I'm iteration 10
Hello world! I'm iteration 11
Hello world! I'm iteration 12
Hello world! I'm iteration 13
Hello world! I'm iteration 14
Hello world! I'm iteration 15
That's all!


### CUDA Hello World example

In [ ]:
 %%sh 
cat > hello.cu << EOF
#include <stdio.h>
 
#define NUM_BLOCKS 16
#define BLOCK_SIZE 1
 
__global__ void hello()
{
    int idx = blockIdx.x;
    printf("Hello world! I'm a thread in block %d\n", idx);
}
 
 
int main(int argc,char **argv)
{
    // launch the kernel
    hello<<<NUM_BLOCKS, BLOCK_SIZE>>>();
 
    // force the printf()s to flush
    cudaDeviceSynchronize();
 
    printf("That's all!\n");
 
    return 0;
}
EOF

In [ ]:
 !nvcc -o hello_cuda hello.cu && ./hello_cuda

Hello world! I'm a thread in block 15
Hello world! I'm a thread in block 9
Hello world! I'm a thread in block 6
Hello world! I'm a thread in block 3
Hello world! I'm a thread in block 0
Hello world! I'm a thread in block 1
Hello world! I'm a thread in block 13
Hello world! I'm a thread in block 7
Hello world! I'm a thread in block 10
Hello world! I'm a thread in block 5
Hello world! I'm a thread in block 8
Hello world! I'm a thread in block 12
Hello world! I'm a thread in block 2
Hello world! I'm a thread in block 11
Hello world! I'm a thread in block 14
Hello world! I'm a thread in block 4
That's all!


### OpenCL Hello World example

In [ ]:
%%sh 
cat > hello.cl << EOF
__kernel void hello() {
    int gid = get_global_id(0);
    printf("Hello world! I'm a thread in block %d\n", gid);
}
EOF

In [ ]:
 %%sh 
cat > hello_CL.c << EOF
#include <CL/cl.h>

#include <stdio.h>
#include <stdlib.h>

#define MAX_SOURCE_SIZE (0x100000)

#define GLOBAl_SIZE 16
#define LOCAL_SIZE 1

int main(int argc, char ** argv) {

	// Load kernel from file hello.cl

	FILE *kernelFile;
	char *kernelSource;
	size_t kernelSize;

	kernelFile = fopen("hello.cl", "r");

	if (!kernelFile) {

		fprintf(stderr, "No file named hello.cl was found\n");

		exit(-1);

	}
	kernelSource = (char*)malloc(MAX_SOURCE_SIZE);
	kernelSize = fread(kernelSource, 1, MAX_SOURCE_SIZE, kernelFile);
	fclose(kernelFile);

	// Getting platform and device information
	cl_platform_id platformId = NULL;
	cl_device_id deviceID = NULL;
	cl_uint retNumDevices;
	cl_uint retNumPlatforms;
	cl_int ret = clGetPlatformIDs(1, &platformId, &retNumPlatforms);
	ret = clGetDeviceIDs(platformId, CL_DEVICE_TYPE_DEFAULT, 1, &deviceID, &retNumDevices);

	// Creating context.
	cl_context context = clCreateContext(NULL, 1, &deviceID, NULL, NULL,  &ret);

	// Creating command queue
	cl_command_queue commandQueue = clCreateCommandQueue(context, deviceID, 0, &ret);

	// Create program from kernel source
	cl_program program = clCreateProgramWithSource(context, 1, (const char **)&kernelSource, (const size_t *)&kernelSize, &ret);	

	// Build program
	ret = clBuildProgram(program, 1, &deviceID, NULL, NULL, NULL);

	// Create kernel
	cl_kernel kernel = clCreateKernel(program, "hello", &ret);

	// Execute the kernel
	size_t globalItemSize = GLOBAl_SIZE;
	size_t localItemSize = LOCAL_SIZE;
	ret = clEnqueueNDRangeKernel(commandQueue, kernel, 1, NULL, &globalItemSize, &localItemSize, 0, NULL, NULL);

  printf("That's all!\n");		

	// Clean up, release memory.
	ret = clFlush(commandQueue);
	ret = clFinish(commandQueue);
	ret = clReleaseCommandQueue(commandQueue);
	ret = clReleaseKernel(kernel);
	ret = clReleaseProgram(program);	
	ret = clReleaseContext(context);

	return 0;
}
EOF

In [ ]:
 !nvcc -o hello_CL hello_CL.c -lOpenCL && ./hello_CL

Hello world! I'm a thread in block 12
Hello world! I'm a thread in block 6
Hello world! I'm a thread in block 0
Hello world! I'm a thread in block 15
Hello world! I'm a thread in block 9
Hello world! I'm a thread in block 3
Hello world! I'm a thread in block 13
Hello world! I'm a thread in block 7
Hello world! I'm a thread in block 1
Hello world! I'm a thread in block 10
Hello world! I'm a thread in block 4
Hello world! I'm a thread in block 14
Hello world! I'm a thread in block 8
Hello world! I'm a thread in block 2
Hello world! I'm a thread in block 11
Hello world! I'm a thread in block 5
That's all!


## Example 2: Vector addition

### C Vector addition

In [ ]:
%%sh 
cat > vector_add_cpu.c << EOF
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <time.h>

#define N 2020
#define MAX_ERR 1e-6

int main(){
    double *a, *b, *out;
    double *d_a, *d_b, *d_out; 

    // Allocate host memory
    a   = (double*)malloc(sizeof(double) * N);
    b   = (double*)malloc(sizeof(double) * N);
    out = (double*)malloc(sizeof(double) * N);

    // Initialize host arrays
    for(int i = 0; i < N; i++){
        a[i] = i / 100.0;
        b[i] = (N - i) / 100.0;
    }

    for(int i = 0; i < N; i++){
        out[i] = a[i] + b[i];
    }   

    // Verification
    for(int i = 0; i < N; i++){
        assert(fabs(out[i] - a[i] - b[i]) < MAX_ERR);
        if (i % 101 == 0)
          printf("%.2f + %.2f = %.2f\n", a[i], b[i], out[i]);
    }
    printf("PASSED\n");

    // Deallocate host memory
    free(a); 
    free(b); 
    free(out);
}
EOF

In [ ]:
!gcc -o vector_add_cpu vector_add_cpu.c && ./vector_add_cpu

0.00 + 20.20 = 20.20
1.01 + 19.19 = 20.20
2.02 + 18.18 = 20.20
3.03 + 17.17 = 20.20
4.04 + 16.16 = 20.20
5.05 + 15.15 = 20.20
6.06 + 14.14 = 20.20
7.07 + 13.13 = 20.20
8.08 + 12.12 = 20.20
9.09 + 11.11 = 20.20
10.10 + 10.10 = 20.20
11.11 + 9.09 = 20.20
12.12 + 8.08 = 20.20
13.13 + 7.07 = 20.20
14.14 + 6.06 = 20.20
15.15 + 5.05 = 20.20
16.16 + 4.04 = 20.20
17.17 + 3.03 = 20.20
18.18 + 2.02 = 20.20
19.19 + 1.01 = 20.20
PASSED


### CUDA Vector addition

In [ ]:
 %%sh 
cat > vector_add_cuda.cu << EOF
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <cuda.h>
#include <cuda_runtime.h>

#define N 2020
#define MAX_ERR 1e-6

__global__ void vector_add(double *out, double *a, double *b, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if(i < n)
        out[i] = a[i] + b[i];
}

int main(){
    double *a, *b, *out;
    double *d_a, *d_b, *d_out; 

    // Allocate host memory
    a   = (double*)malloc(sizeof(double) * N);
    b   = (double*)malloc(sizeof(double) * N);
    out = (double*)malloc(sizeof(double) * N);

    // Initialize host arrays
    for(int i = 0; i < N; i++){
        a[i] = i / 100.0;
        b[i] = (N - i) / 100.0;
    }

    // Allocate device memory
    cudaMalloc((void**)&d_a, sizeof(double) * N);
    cudaMalloc((void**)&d_b, sizeof(double) * N);
    cudaMalloc((void**)&d_out, sizeof(double) * N);

    // Transfer data from host to device memory
    cudaMemcpy(d_a, a, sizeof(double) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(double) * N, cudaMemcpyHostToDevice);

    // Executing kernel
    int threadsPerBlock = 1024;
    //int blocksPerGrid =(N + threadsPerBlock - 1) / threadsPerBlock;
    int blocksPerGrid = N/threadsPerBlock + (N % threadsPerBlock == 0 ? 0:1);
    vector_add<<<blocksPerGrid, threadsPerBlock>>>(d_out, d_a, d_b, N);
    
    // Transfer data back to host memory
    cudaMemcpy(out, d_out, sizeof(double) * N, cudaMemcpyDeviceToHost);
     
    // Verification
    for(int i = 0; i < N; i++){
        assert(fabs(out[i] - a[i] - b[i]) < MAX_ERR);
        if (i % 101 == 0)
          printf("%.2f + %.2f = %.2f\n", a[i], b[i], out[i]);
    }
    printf("PASSED\n");

    // Deallocate device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);

    // Deallocate host memory
    free(a); 
    free(b); 
    free(out);
}
EOF

In [ ]:
 !nvcc -o vector_add_cuda vector_add_cuda.cu && ./vector_add_cuda

0.00 + 20.20 = 20.20
1.01 + 19.19 = 20.20
2.02 + 18.18 = 20.20
3.03 + 17.17 = 20.20
4.04 + 16.16 = 20.20
5.05 + 15.15 = 20.20
6.06 + 14.14 = 20.20
7.07 + 13.13 = 20.20
8.08 + 12.12 = 20.20
9.09 + 11.11 = 20.20
10.10 + 10.10 = 20.20
11.11 + 9.09 = 20.20
12.12 + 8.08 = 20.20
13.13 + 7.07 = 20.20
14.14 + 6.06 = 20.20
15.15 + 5.05 = 20.20
16.16 + 4.04 = 20.20
17.17 + 3.03 = 20.20
18.18 + 2.02 = 20.20
19.19 + 1.01 = 20.20
PASSED


### OpenCL Vector addition

In [ ]:
%%sh 
cat > vector_add.cl << EOF
__kernel void vector_add(__global double *a, __global double *b, __global double *out, int n) {
    
    int i = get_global_id(0);

    if(i < n)
      out[i] = a[i] + b[i];
}
EOF

In [ ]:
%%sh 
cat > vector_add_opencl.c << EOF
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>

#ifdef __APPLE__
#include <OpenCL/opencl.h>
#else
#include <CL/cl.h>
#endif

#define MAX_SOURCE_SIZE (0x100000)
#define N 2020
#define MAX_ERR 1e-6

int main(void) {
    // Create the two input vectors
    double *a, *b, *out;
    int n = N;
    
    // Allocate host memory
    a   = (double*)malloc(sizeof(double) * N);
    b   = (double*)malloc(sizeof(double) * N);
    out = (double*)malloc(sizeof(double) * N);

    // Initialize host arrays
    for(int i = 0; i < N; i++){
        a[i] = i / 100.0;
        b[i] = (N - i) / 100.0;
    }


    // Load the kernel source code into the array source_str
    FILE *fp;
    char *source_str;
    size_t source_size;

    fp = fopen("vector_add.cl", "r");
    if (!fp) {
        fprintf(stderr, "Failed to load kernel.\n");
        exit(1);
    }
    source_str = (char*)malloc(MAX_SOURCE_SIZE);
    source_size = fread( source_str, 1, MAX_SOURCE_SIZE, fp);
    fclose( fp );

    // Get platform and device information
    cl_platform_id platform_id = NULL;
    cl_device_id device_id = NULL;   
    cl_uint ret_num_devices;
    cl_uint ret_num_platforms;
    cl_int ret = clGetPlatformIDs(1, &platform_id, &ret_num_platforms);
    ret = clGetDeviceIDs( platform_id, CL_DEVICE_TYPE_ALL, 1, 
            &device_id, &ret_num_devices);

    // Create an OpenCL context
    cl_context context = clCreateContext( NULL, 1, &device_id, NULL, NULL, &ret);

    // Create a command queue
    cl_command_queue command_queue = clCreateCommandQueue(context, device_id, 0, &ret);

    // Create memory buffers on the device for each vector 
    cl_mem a_mem_obj = clCreateBuffer(context, CL_MEM_READ_ONLY, 
            N * sizeof(double), NULL, &ret);
    cl_mem b_mem_obj = clCreateBuffer(context, CL_MEM_READ_ONLY,
            N * sizeof(double), NULL, &ret);
    cl_mem out_mem_obj = clCreateBuffer(context, CL_MEM_WRITE_ONLY, 
            N * sizeof(double), NULL, &ret);

    // Copy the lists a and b to their respective memory buffers
    ret = clEnqueueWriteBuffer(command_queue, a_mem_obj, CL_TRUE, 0,
            N * sizeof(double), a, 0, NULL, NULL);
    ret = clEnqueueWriteBuffer(command_queue, b_mem_obj, CL_TRUE, 0, 
            N * sizeof(double), b, 0, NULL, NULL);

    // Create a program from the kernel source
    cl_program program = clCreateProgramWithSource(context, 1, 
            (const char **)&source_str, (const size_t *)&source_size, &ret);

    // Build the program
    ret = clBuildProgram(program, 1, &device_id, NULL, NULL, NULL);

    // Create the OpenCL kernel
    cl_kernel kernel = clCreateKernel(program, "vector_add", &ret);

    // Set the arguments of the kernel
    ret = clSetKernelArg(kernel, 0, sizeof(cl_mem), (void *)&a_mem_obj);
    ret = clSetKernelArg(kernel, 1, sizeof(cl_mem), (void *)&b_mem_obj);
    ret = clSetKernelArg(kernel, 2, sizeof(cl_mem), (void *)&out_mem_obj);
    ret = clSetKernelArg(kernel, 3, sizeof(cl_int), (void *)&n);
    
    // Execute the OpenCL kernel on the list

    size_t local_item_size = 64; // Process in groups of 64
    int n_blocks = n/local_item_size + (n % local_item_size == 0 ? 0:1);
    size_t global_item_size = n_blocks * local_item_size;
    
    ret = clEnqueueNDRangeKernel(command_queue, kernel, 1, NULL, 
            &global_item_size, &local_item_size, 0, NULL, NULL);

    // Read the memory buffer C on the device to the local variable C
    //int *C = (int*)malloc(sizeof(int) * N);
    ret = clEnqueueReadBuffer(command_queue, out_mem_obj, CL_TRUE, 0, 
            N * sizeof(double), out, 0, NULL, NULL);

    // Display the result to the screen
    for(int i = 0; i < N; i++){
        assert(fabs(out[i] - a[i] - b[i]) < MAX_ERR);
        if (i % 101 == 0)
          printf("%.2f + %.2f = %.2f\n", a[i], b[i], out[i]);
    }
    printf("PASSED\n");

    // Clean up
    ret = clFlush(command_queue);
    ret = clFinish(command_queue);
    ret = clReleaseKernel(kernel);
    ret = clReleaseProgram(program);
    ret = clReleaseMemObject(a_mem_obj);
    ret = clReleaseMemObject(b_mem_obj);
    ret = clReleaseMemObject(out_mem_obj);
    ret = clReleaseCommandQueue(command_queue);
    ret = clReleaseContext(context);
    free(a);
    free(b);
    free(out);
    return 0;
}
EOF

In [ ]:
 !nvcc -o vector_add_opencl vector_add_opencl.c -lOpenCL && ./vector_add_opencl

0.00 + 20.20 = 20.20
1.01 + 19.19 = 20.20
2.02 + 18.18 = 20.20
3.03 + 17.17 = 20.20
4.04 + 16.16 = 20.20
5.05 + 15.15 = 20.20
6.06 + 14.14 = 20.20
7.07 + 13.13 = 20.20
8.08 + 12.12 = 20.20
9.09 + 11.11 = 20.20
10.10 + 10.10 = 20.20
11.11 + 9.09 = 20.20
12.12 + 8.08 = 20.20
13.13 + 7.07 = 20.20
14.14 + 6.06 = 20.20
15.15 + 5.05 = 20.20
16.16 + 4.04 = 20.20
17.17 + 3.03 = 20.20
18.18 + 2.02 = 20.20
19.19 + 1.01 = 20.20
PASSED


## Numerical integration (Riemann sum): calculating $\Phi(1) = \frac 1 {\sqrt{2\pi}} \int_{0}^1 e^{-x^2/2} \, dx$
(see, e.g.: https://mathworld.wolfram.com/NormalDistributionFunction.html).

### C Numerical integration (Riemann sum) with serial code on CPU

In [ ]:
%%sh 
cat > riemann_cpu_double.c << EOF
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

#include <string.h>
#include <stdbool.h>

#define N 1000000000

double riemann(int n)
{
  double sum = 0;
  for(int i = 0; i < n; ++i)
  {
    double x = (double) i / (double) n;
    double fx = (exp(-x * x / 2.0) + exp(-(x + 1 / (double)n) * (x + 1 / (double)n) / 2.0)) / 2.0;
    sum += fx;
  }
  sum *= (1.0 / sqrt(2.0 * M_PI)) / (double) n;
  return sum;
}

int main(int argc, char** argv){

  clock_t t1; 
  t1 = clock();

  double sum = riemann(N);

  t1 = clock() - t1;

  double time_taken1 = ((double)t1)/CLOCKS_PER_SEC; // in seconds

  printf("Riemann sum CPU (double precision) for N = %d     : %.17g \n", N, sum);
  printf("Total time (measured by CPU)                              : %f s\n", time_taken1);
} 
EOF

#### Compilation without optimization

In [ ]:
!g++ -o riemann_cpu_double riemann_cpu_double.c && ./riemann_cpu_double

Riemann sum CPU (double precision) for N = 1000000000     : 0.3413447460685729 
Total time (measured by CPU)                              : 92.912311 s


#### Compilation with -O3 optimization

In [ ]:
 !g++ -O3 -o riemann_cpu_double_o3 riemann_cpu_double.c && ./riemann_cpu_double_o3

Riemann sum CPU (double precision) for N = 1000000000     : 0.3413447460685729 
Total time (measured by CPU)                              : 39.254424 s


### Version with one kernel (trapezoid median)

#### CUDA version (trapezoid median)

In [ ]:
%%sh
cat > riemann_cuda_double.cu << EOF
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
 
#define N 1000000000
 
/* CUDA error wraper */
static void CUDA_ERROR( cudaError_t err) 
{
    if (err != cudaSuccess) {
        printf("CUDA ERROR: %s, exiting\n", cudaGetErrorString(err));
        exit(-1);
    }
}
 
__global__ void medianTrapezoid(double *a, int n)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  double x = (double)idx / (double)n;
 
  if(idx < n)
    a[idx] = (exp(-x * x / 2.0) + exp(-(x + 1 / (double)n) * (x + 1 / (double)n) / 2.0)) / 2.0;
}
 
double riemannCUDA(int n)
{
  ///size of the arrays in bytes
  size_t size = n * sizeof(double);
 
  // allocate array on host and device
  double* a_h = (double *)malloc(size);
  double* a_d; cudaMalloc((double **) &a_d, size);
 
  // do calculation on device
  int block_size = 1024;
  int n_blocks = n/block_size + (n % block_size == 0 ? 0:1);
  printf("CUDA kernel 'medianTrapezoid' launch with %d blocks of %d threads\n", n_blocks, block_size);
  medianTrapezoid <<< n_blocks, block_size >>> (a_d, n);
  
  // copy results from device to host
  cudaMemcpy(a_h, a_d, sizeof(double)*n, cudaMemcpyDeviceToHost);
 
  // add up results
  double sum = 0;
  for (int i=0; i < n; i++) sum += a_h[i];
  sum *= (1.0 / sqrt(2.0 * M_PI)) / (double)n;
  
  // clean up
  free(a_h); cudaFree(a_d);
  
  return sum;
}
 
int main(int argc, char** argv){
 
  /*get info on our GPU, defaulting to first one*/
  cudaDeviceProp prop;
  CUDA_ERROR(cudaGetDeviceProperties(&prop,0));
  printf("Found GPU '%s' with %g GB of global memory, max %d threads per block, and %d multiprocessors\n", 
         prop.name, prop.totalGlobalMem/(1024.0*1024.0*1024.0),
         prop.maxThreadsPerBlock,prop.multiProcessorCount);
 
  /*init CUDA*/
  CUDA_ERROR(cudaSetDevice(0));
 
  clock_t t1; 
  t1 = clock();
 
  double sum = riemannCUDA(N);
 
  t1 = clock() - t1;
 
  double time_taken1 = ((double)t1)/CLOCKS_PER_SEC; // in seconds
 
  printf("Riemann sum CUDA (double precision) for N = %d    : %.17g \n", N, sum);
  printf("Total time (measured by CPU)                              : %f s\n", time_taken1);
} 
EOF

In [ ]:
!nvcc -o riemann_cuda_double riemann_cuda_double.cu && ./riemann_cuda_double

Found GPU 'Tesla P100-PCIE-16GB' with 15.8993 GB of global memory, max 1024 threads per block, and 56 multiprocessors
tcmalloc: large alloc 8000004096 bytes == 0x55a84cbda000 @  0x7f683b14e1e7 0x55a84b4dab65 0x55a84b4dadb6 0x7f683a17fb97 0x55a84b4da9ea
CUDA kernel 'medianTrapezoid' launch with 976563 blocks of 1024 threads
Riemann sum CUDA (double precision) for N = 1000000000    : 0.3413447460685729 
Total time (measured by CPU)                              : 9.069266 s


#### CUDA profiling (trapezoid median)

In [ ]:
!nvprof ./riemann_cuda_double

==673== NVPROF is profiling process 673, command: ./riemann_cuda_double
Found GPU 'Tesla P100-PCIE-16GB' with 15.8993 GB of global memory, max 1024 threads per block, and 56 multiprocessors
tcmalloc: large alloc 8000004096 bytes == 0x56106fbd6000 @  0x7f0b41be71e7 0x56106c4c2b65 0x56106c4c2db6 0x7f0b40c18b97 0x56106c4c29ea
CUDA kernel 'medianTrapezoid' launch with 976563 blocks of 1024 threads
Riemann sum CUDA (double precision) for N = 1000000000    : 0.3413447460685729 
Total time (measured by CPU)                              : 9.000705 s
==673== Profiling application: ./riemann_cuda_double
==673== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.35%  5.27062s         1  5.27062s  5.27062s  5.27062s  [CUDA memcpy DtoH]
                    0.65%  34.510ms         1  34.510ms  34.510ms  34.510ms  medianTrapezoid(double*, int)
      API calls:   96.67%  5.30608s         1  5.30608s  5.30608s  5.30608s  cudaMemcpy


#### OpenCL version (trapezoid median)

In [ ]:
%%sh 
cat > riemann.cl << EOF
__kernel void medianTrapezoid(__global double *a, int n) {
    
    int idx = get_global_id(0);
    double x = (double)idx / (double)n;
 
    if(idx < n)
       a[idx] = (exp(-x * x / 2.0) + exp(-(x + 1 / (double)n) * (x + 1 / (double)n) / 2.0)) / 2.0;
}
EOF

In [ ]:
%%sh 
cat > riemann_opencl_double.c << EOF
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>
#include <CL/cl.h>

#define MAX_SOURCE_SIZE (0x100000)

#define N 1000000000

double riemannCL(int n)
{
    //Allocate memory to host variable
    double *a = (double*)malloc(sizeof(double)*n);
    
    // Load the kernel source code into the array source_str
    FILE *fp;
    char *source_str;
    size_t source_size;

    fp = fopen("riemann.cl", "r");
    if (!fp) {
        fprintf(stderr, "Failed to load kernel.\n");
        exit(1);
    }
    source_str = (char*)malloc(MAX_SOURCE_SIZE);
    source_size = fread( source_str, 1, MAX_SOURCE_SIZE, fp);
    fclose( fp );

    // Get platform and device information
    cl_platform_id platform_id = NULL;
    cl_device_id device_id = NULL;   
    cl_uint ret_num_devices;
    cl_uint ret_num_platforms;
    cl_int ret = clGetPlatformIDs(1, &platform_id, &ret_num_platforms);
    ret = clGetDeviceIDs( platform_id, CL_DEVICE_TYPE_ALL, 1, 
            &device_id, &ret_num_devices);

    // Create an OpenCL context
    cl_context context = clCreateContext( NULL, 1, &device_id, NULL, NULL, &ret);

    // Create a command queue
    cl_command_queue command_queue = clCreateCommandQueue(context, device_id, 0, &ret);

    // Create memory buffers on the device for each vector 
    cl_mem a_mem_obj = clCreateBuffer(context, CL_MEM_READ_WRITE, 
            n * sizeof(double), NULL, &ret);

    // Create a program from the kernel source
    cl_program program = clCreateProgramWithSource(context, 1, 
            (const char **)&source_str, (const size_t *)&source_size, &ret);

    // Build the program
    ret = clBuildProgram(program, 1, &device_id, NULL, NULL, NULL);

    clock_t t2; 
    t2 = clock(); 

    // Create the OpenCL kernel
    cl_kernel kernel = clCreateKernel(program, "medianTrapezoid", &ret);

    // Set the arguments of the kernel
    ret = clSetKernelArg(kernel, 0, sizeof(cl_mem), (void *)&a_mem_obj);    
    ret = clSetKernelArg(kernel, 1, sizeof(cl_int), (void *)&n);
    
    // Execute the OpenCL kernel
    size_t local_item_size = 1024;
    int n_blocks = n/local_item_size + (n % local_item_size == 0 ? 0:1);
    size_t global_item_size = n_blocks * local_item_size;
    printf("OpenCL kernel 'medianTrapezoid' launch with %d blocks of %lu threads\n\n", n_blocks, local_item_size);

    ret = clEnqueueNDRangeKernel(command_queue, kernel, 1, NULL, 
            &global_item_size, &local_item_size, 0, NULL, NULL);

    t2 = clock() - t2;

    double time_taken2 = ((double)t2)/CLOCKS_PER_SEC; // in seconds

    clock_t t3; 
    t3 = clock();

    ret = clEnqueueReadBuffer(command_queue, a_mem_obj, CL_TRUE, 0, 
            n * sizeof(double), a, 0, NULL, NULL);

    t3 = clock() - t3;

    double time_taken3 = ((double)t3)/CLOCKS_PER_SEC; // in seconds

    clock_t t4; 
    t4 = clock(); 

    // add up results
    double sum = 0;
    for (int i=0; i < n; i++) sum += a[i];
    sum *= (1.0 / sqrt(2.0 * M_PI)) / (double)n;

    t4 = clock() - t4;

    double time_taken4 = ((double)t4)/CLOCKS_PER_SEC; // in seconds

    // Clean up
    ret = clFlush(command_queue);
    ret = clFinish(command_queue);
    ret = clReleaseKernel(kernel);
    ret = clReleaseProgram(program);
    ret = clReleaseMemObject(a_mem_obj); 
    ret = clReleaseCommandQueue(command_queue);
    ret = clReleaseContext(context);
    free(a);

    printf("OpenCL and CPU code diagnostics:\n");
    printf("OpenCL kernel execution time (measured by CPU):        %f ms\n", time_taken2 * 1000);
    printf("Device to host memory transfer time (measured by CPU): %f s\n", time_taken3);
    printf("CPU execution time for adding sum (measured by CPU):   %f s\n\n", time_taken4);
  
    return sum;
}

int main(int argc, char** argv){

  clock_t t1; 
  t1 = clock(); 

  double sum = riemannCL(N);

  t1 = clock() - t1;

  double time_taken1 = ((double)t1)/CLOCKS_PER_SEC; // in seconds

  printf("Riemann sum OpenCL (double precision) for N = %d    : %.17g \n", N, sum);
  printf("Total time (measured by CPU)                                : %f s\n", time_taken1);
}
EOF

In [ ]:
!nvcc -o riemann_opencl_double riemann_opencl_double.c -lOpenCL && ./riemann_opencl_double

tcmalloc: large alloc 8000004096 bytes == 0x556e96f94000 @  0x7f19f26d71e7 0x556e95616074 0x556e95616604 0x7f19f1aa2b97 0x556e95615f4a
OpenCL kernel 'medianTrapezoid' launch with 976563 blocks of 1024 threads

OpenCL and CPU code diagnostics:
OpenCL kernel execution time (measured by CPU):        8.093000 ms
Device to host memory transfer time (measured by CPU): 5.294761 s
CPU execution time for adding sum (measured by CPU):   3.335286 s

Riemann sum OpenCL (double precision) for N = 1000000000    : 0.3413447460685729 
Total time (measured by CPU)                                : 9.266874 s


### Version with two kernels (trapezoid median + sum reducer)

#### CUDA version (trapezoid median + sum reducer)

In [ ]:
%%sh
cat > riemann_cuda_double_reduce.cu << EOF
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

#define N 1000000000

/* CUDA error wraper */
static void CUDA_ERROR( cudaError_t err) 
{
    if (err != cudaSuccess) {
        printf("CUDA ERROR: %s, exiting\n", cudaGetErrorString(err));
        exit(-1);
    }
}

__global__ void medianTrapezoid(double *a, int n)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  double x = (double)idx / (double)n;
 
  if(idx < n)
    a[idx] = (exp(-x * x / 2.0) + exp(-(x + 1 / (double)n) * (x + 1 / (double)n) / 2.0)) / 2.0;
}

__global__ void reducerSum(double *a, double *out, int n, int block_size) {
    int idx = threadIdx.x;
    double sum = 0;
    for (int i = idx; i < n; i += block_size)
        sum += a[i];
    extern __shared__ double r[];
    r[idx] = sum;
    __syncthreads();
    for (int size = block_size/2; size>0; size/=2) {
        if (idx<size)
            r[idx] += r[idx+size];
        __syncthreads();
    }
    if (idx == 0)
        *out = r[0];
}

double riemannCUDA(int n)
{
  ///size of the arrays in bytes
  size_t size = n * sizeof(double);

  int block_size = 1024;

  // allocate array on host and device
  double* a_h = (double *)malloc(size);
  double* out_h = (double *)malloc(sizeof(double));
  double* r = (double *)malloc(block_size * sizeof(double));
  double* a_d; cudaMalloc((double **) &a_d, size);
  double* out; cudaMalloc((double **) &out, sizeof(double));

  // do calculation on device
  
  int n_blocks = n/block_size + (n % block_size == 0 ? 0:1);
  printf("CUDA kernel 'medianTrapezoid' launch with %d blocks of %d threads\n", n_blocks, block_size);
  medianTrapezoid <<< n_blocks, block_size >>> (a_d, n);
  int n_blocks2 = 1;
  printf("CUDA kernel 'reducerSum' launch with %d blocks of %d threads\n\n", n_blocks2, block_size);
  reducerSum <<< n_blocks2, block_size, block_size*sizeof(double) >>> (a_d, out, n, block_size);
  
  // copy results from device to host
  cudaMemcpy(out_h, out, sizeof(double), cudaMemcpyDeviceToHost);

  // add up results
  double sum;
  sum = *out_h;
  sum *= (1.0 / sqrt(2.0 * M_PI)) / (double)n;
  
  // clean up
  free(a_h); cudaFree(a_d);
  free(out_h); cudaFree(out);
  cudaFree(r);
  
  return sum;
}


int main(int argc, char** argv){

  /*get info on our GPU, defaulting to first one*/
  cudaDeviceProp prop;
  CUDA_ERROR(cudaGetDeviceProperties(&prop,0));
  printf("Found GPU '%s' with %g GB of global memory, max %d threads per block, and %d multiprocessors\n", 
         prop.name, prop.totalGlobalMem/(1024.0*1024.0*1024.0),
         prop.maxThreadsPerBlock,prop.multiProcessorCount);
 
  /*init CUDA*/
  CUDA_ERROR(cudaSetDevice(0));

  clock_t t1; 
  t1 = clock();

  double sum = riemannCUDA(N);

  t1 = clock() - t1;

  double time_taken1 = ((double)t1)/CLOCKS_PER_SEC; // in seconds

  printf("Riemann sum CUDA (double precision) for N = %d    : %.17g \n", N, sum);
  printf("Total time (measured by CPU)                              : %f s\n", time_taken1);
} 
EOF

In [ ]:
!nvcc -o riemann_cuda_double_reduce riemann_cuda_double_reduce.cu && ./riemann_cuda_double_reduce

Found GPU 'Tesla P100-PCIE-16GB' with 15.8993 GB of global memory, max 1024 threads per block, and 56 multiprocessors
tcmalloc: large alloc 8000004096 bytes == 0x556fe8472000 @  0x7fd1fe2a21e7 0x556fe6897b76 0x556fe6897e96 0x7fd1fd2d3b97 0x556fe68979ea
CUDA kernel 'medianTrapezoid' launch with 976563 blocks of 1024 threads
CUDA kernel 'reducerSum' launch with 1 blocks of 1024 threads

Riemann sum CUDA (double precision) for N = 1000000000    : 0.34134474606854243 
Total time (measured by CPU)                              : 0.608075 s


#### CUDA profiling (trapezoid median + sum reducer)

In [ ]:
!nvprof ./riemann_cuda_double_reduce

==771== NVPROF is profiling process 771, command: ./riemann_cuda_double_reduce
Found GPU 'Tesla P100-PCIE-16GB' with 15.8993 GB of global memory, max 1024 threads per block, and 56 multiprocessors
tcmalloc: large alloc 8000004096 bytes == 0x5572de062000 @  0x7f75efc9f1e7 0x5572da7f4b76 0x5572da7f4e96 0x7f75eecd0b97 0x5572da7f49ea
CUDA kernel 'medianTrapezoid' launch with 976563 blocks of 1024 threads
CUDA kernel 'reducerSum' launch with 1 blocks of 1024 threads

Riemann sum CUDA (double precision) for N = 1000000000    : 0.34134474606854243 
Total time (measured by CPU)                              : 0.639958 s
==771== Profiling application: ./riemann_cuda_double_reduce
==771== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   93.08%  420.01ms         1  420.01ms  420.01ms  420.01ms  reducerSum(double*, double*, int, int)
                    6.92%  31.212ms         1  31.212ms  31.212ms  31.212ms  medianTrapezoid(doub

#### OpenCL version (trapezoid median + sum reducer)

In [ ]:
%%sh 
cat > riemann_reduce.cl << EOF
__kernel void medianTrapezoid(__global double *a, int n) {
    
    int idx = get_global_id(0);
    double x = (double)idx / (double)n;
 
    if(idx < n)
       a[idx] = (exp(-x * x / 2.0) + exp(-(x + 1 / (double)n) * (x + 1 / (double)n) / 2.0)) / 2.0;
}

__kernel void reducerSum(__global double *a, __global double *out, __local double *r, int n, int block_size)
{
    int idx = get_local_id(0);
    double sum = 0;
    for (int i = idx; i < n; i += block_size)
        sum += a[i];
    r[idx] = sum;
    barrier(CLK_LOCAL_MEM_FENCE);

    for (int size = block_size/2; size>0; size/=2) {
        if (idx<size)
            r[idx] += r[idx+size];
        barrier(CLK_LOCAL_MEM_FENCE);
    }
   
    if (idx == 0)
        *out = r[0];
}
EOF

In [ ]:
%%sh 
cat > riemann_opencl_double_reduce.c << EOF
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>
#include <CL/cl.h>

#define MAX_SOURCE_SIZE (0x100000)

#define N 1000000000

double riemannCL(int n)
{
    //Allocate memory to host variable
    int block_size = 1024;
    double *a = (double*)malloc(sizeof(double) * n);
    double *out = (double*)malloc(sizeof(double));
    
    // Load the kernel source code into the array source_str
    FILE *fp;
    char *source_str;
    size_t source_size;

    fp = fopen("riemann_reduce.cl", "r");
    if (!fp) {
        fprintf(stderr, "Failed to load kernel.\n");
        exit(1);
    }
    source_str = (char*)malloc(MAX_SOURCE_SIZE);
    source_size = fread( source_str, 1, MAX_SOURCE_SIZE, fp);
    fclose( fp );

    // Get platform and device information
    cl_platform_id platform_id = NULL;
    cl_device_id device_id = NULL;   
    cl_uint ret_num_devices;
    cl_uint ret_num_platforms;
    cl_int ret = clGetPlatformIDs(1, &platform_id, &ret_num_platforms);
    ret = clGetDeviceIDs( platform_id, CL_DEVICE_TYPE_ALL, 1, 
            &device_id, &ret_num_devices);

    // Create an OpenCL context
    cl_context context = clCreateContext( NULL, 1, &device_id, NULL, NULL, &ret);

    // Create a command queue
    cl_command_queue command_queue = clCreateCommandQueue(context, device_id, 0, &ret);

    // Create memory buffers on the device for each vector 
    cl_mem a_mem_obj = clCreateBuffer(context, CL_MEM_READ_WRITE, 
            n * sizeof(double), NULL, &ret);
    cl_mem out_mem_obj = clCreateBuffer(context, CL_MEM_READ_WRITE, 
            sizeof(double), NULL, &ret);

    // Create a program from the kernel source
    cl_program program = clCreateProgramWithSource(context, 1, 
            (const char **)&source_str, (const size_t *)&source_size, &ret);

    // Build the program
    ret = clBuildProgram(program, 1, &device_id, NULL, NULL, NULL);

    clock_t t2; 
    t2 = clock(); 

    // Create the OpenCL kernel
    cl_kernel kernel = clCreateKernel(program, "medianTrapezoid", &ret);

    // Set the arguments of the kernel
    ret = clSetKernelArg(kernel, 0, sizeof(cl_mem), (void *)&a_mem_obj);    
    ret = clSetKernelArg(kernel, 1, sizeof(cl_int), (void *)&n);
    
    // Execute the OpenCL kernel
    size_t local_item_size = block_size;
    int n_blocks = n/local_item_size + (n % local_item_size == 0 ? 0:1);
    size_t global_item_size = n_blocks * local_item_size;
    printf("OpenCL kernel 'medianTrapezoid' launch with %d blocks of %lu threads\n", n_blocks, local_item_size);

    ret = clEnqueueNDRangeKernel(command_queue, kernel, 1, NULL, 
            &global_item_size, &local_item_size, 0, NULL, NULL);

    // Create the OpenCL kernel2
    cl_kernel kernel2 = clCreateKernel(program, "reducerSum", &ret);

    // Set the arguments of the kernel2
    ret = clSetKernelArg(kernel2, 0, sizeof(cl_mem), (void *)&a_mem_obj);    
    ret = clSetKernelArg(kernel2, 1, sizeof(cl_mem), (void *)&out_mem_obj);   
    ret = clSetKernelArg(kernel2, 2, block_size * sizeof(cl_double), NULL);    
    ret = clSetKernelArg(kernel2, 3, sizeof(cl_int), (void *)&n);   
    ret = clSetKernelArg(kernel2, 4, sizeof(cl_int), (void *)&block_size);

    // Execute the OpenCL kernel2
    size_t local_item_size2 = block_size;
    size_t global_item_size2 = block_size;
    printf("OpenCL kernel 'reducerSum' launch with %lu blocks of %lu threads\n\n", global_item_size2/local_item_size2, local_item_size2);

    ret = clEnqueueNDRangeKernel(command_queue, kernel2, 1, NULL, 
            &global_item_size2, &local_item_size2, 0, NULL, NULL);

    t2 = clock() - t2;

    double time_taken2 = ((double)t2)/CLOCKS_PER_SEC; // in seconds

    clock_t t3; 
    t3 = clock();

   
    ret = clEnqueueReadBuffer(command_queue, out_mem_obj, CL_TRUE, 0, 
            sizeof(double), out, 0, NULL, NULL);

    t3 = clock() - t3;

    double time_taken3 = ((double)t3)/CLOCKS_PER_SEC; // in seconds

    // add up results
    double sum;
    sum = *out;
    sum *= (1.0 / sqrt(2.0 * M_PI)) / (double)n;

    // Clean up
    ret = clFlush(command_queue);
    ret = clFinish(command_queue);
    ret = clReleaseKernel(kernel);
    ret = clReleaseKernel(kernel2);
    ret = clReleaseProgram(program);
    ret = clReleaseMemObject(a_mem_obj);
    ret = clReleaseMemObject(out_mem_obj);
    ret = clReleaseCommandQueue(command_queue);
    ret = clReleaseContext(context);
    free(a);

    printf("OpenCL and CPU code diagnostics:\n");
    printf("OpenCL kernels execution time (measured by CPU):        %f ms\n", time_taken2 * 1000);
    printf("Device to host memory transfer time (measured by CPU):  %f s\n\n", time_taken3);
  
    return sum;
}

int main(int argc, char** argv){

  clock_t t1; 
  t1 = clock(); 

  double sum = riemannCL(N);

  t1 = clock() - t1;

  double time_taken1 = ((double)t1)/CLOCKS_PER_SEC; // in seconds

  printf("Riemann sum OpenCL (double precision) for N = %d    : %.17g \n", N, sum);
  printf("Total time (measured by CPU)                                : %f s\n", time_taken1);
}
EOF

In [ ]:
!nvcc -o riemann_opencl_double_reduce riemann_opencl_double_reduce.c -lOpenCL && ./riemann_opencl_double_reduce

tcmalloc: large alloc 8000004096 bytes == 0x55fc747ea000 @  0x7feebf5de1e7 0x55fc734c307e 0x55fc734c3748 0x7feebe9a9b97 0x55fc734c2f4a
OpenCL kernel 'medianTrapezoid' launch with 976563 blocks of 1024 threads
OpenCL kernel 'reducerSum' launch with 1 blocks of 1024 threads

OpenCL and CPU code diagnostics:
OpenCL kernels execution time (measured by CPU):        8.269000 ms
Device to host memory transfer time (measured by CPU):  0.450963 s

Riemann sum OpenCL (double precision) for N = 1000000000    : 0.34134474606854243 
Total time (measured by CPU)                                : 0.873289 s


### Python GPU versions with one kernel (trapezoid median)

#### pyCUDA version (trapezoid median)

In [ ]:
pip -q install pycuda

     |████████████████████████████████| 1.6MB 8.4MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 


In [ ]:
from __future__ import print_function
from __future__ import absolute_import
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import time

import numpy

def iDivUp(a, b):
    return a // b + 1

N = 1000000000

def riemannCUDA(n):
    a = numpy.empty([n])

    a = a.astype(numpy.float64)

    a_d = cuda.mem_alloc(a.size * a.dtype.itemsize)

    mod = SourceModule("""
        __global__ void medianTrapezoid(double *a, int n)
        {
          int idx = blockIdx.x * blockDim.x + threadIdx.x;
          double x = (double)idx / (double)n;
 
          if(idx < n)
            a[idx] = (exp(-x * x / 2.0) + exp(-(x + 1 / (double)n) * (x + 1 / (double)n) / 2.0)) / 2.0;
        }
        """)

    func = mod.get_function("medianTrapezoid")
    block_size = 1024
    n_blocks = iDivUp(n, block_size)
    blockDim  = (block_size, 1, 1)
    gridDim   = (n_blocks, 1, 1)
    print("CUDA kernel 'medianTrapezoid' launch with %i blocks of %i threads\n" % (n_blocks, block_size))
    func(a_d, numpy.int32(n), block=blockDim, grid=gridDim)

    cuda.memcpy_dtoh(a, a_d)

    Sum = numpy.sum(a) / numpy.sqrt(2 * numpy.pi) / numpy.float64(n)

    return Sum

dev = pycuda.autoinit.device

dev_name = dev.name()
total_memory = dev.total_memory() / 1024.0 / 1024.0 / 1024.0
threads_per_block = dev.get_attribute(pycuda.driver.device_attribute.MAX_THREADS_PER_BLOCK)
sm_count = dev.get_attribute(pycuda.driver.device_attribute.MULTIPROCESSOR_COUNT)

print("Found GPU '%s' with %.3f GB of global memory, max %i threads per block, and %i multiprocessors\n" % 
       (dev_name, total_memory, threads_per_block, sm_count))

start = time.time()

Sum = riemannCUDA(N)

end = time.time()

time_taken = end - start # in seconds

print("Riemann sum pyCUDA (double precision) for N = %i  : %.17f" % (N, Sum));
print("Total time (measured by CPU)                              : %f s" % time_taken);

Found GPU 'Tesla P100-PCIE-16GB' with 15.899 GB of global memory, max 1024 threads per block, and 56 multiprocessors

CUDA kernel 'medianTrapezoid' launch with 976563 blocks of 1024 threads

Riemann sum pyCUDA (double precision) for N = 1000000000  : 0.34134474606853665
Total time (measured by CPU)                              : 5.707517 s


#### pyOpenCL version (trapezoid median)

In [ ]:
pip -q install pyopencl

     |████████████████████████████████| 727kB 8.6MB/s 


In [ ]:
from __future__ import absolute_import, print_function
import numpy as np
import pyopencl as cl

import time

def iDivUp(a, b):
    return a // b + 1

N = 1000000000

def riemannOpenCL(n):

    a = np.empty([n])
    a = a.astype(np.float64)

    queue = cl.CommandQueue(ctx)

    mf = cl.mem_flags
    a_d = cl.Buffer(ctx, mf.WRITE_ONLY, a.nbytes)

    prg = cl.Program(ctx, """
    __kernel void medianTrapezoid(__global double *a, int n) {
    
        int idx = get_global_id(0);
        double x = (double)idx / (double)n;
 
        if(idx < n)
           a[idx] = (exp(-x * x / 2.0) + exp(-(x + 1 / (double)n) * (x + 1 / (double)n) / 2.0)) / 2.0;
    }
    """).build()

    local_item_size = 1024
    n_blocks = iDivUp(n, local_item_size)
    global_item_size = n_blocks * local_item_size
    print("OpenCL kernel 'medianTrapezoid' launch with %i blocks of %i threads\n" % (n_blocks, local_item_size))
    prg.medianTrapezoid(queue, (global_item_size, 1, 1), (local_item_size, 1, 1), a_d, np.int32(n))

    cl.enqueue_copy(queue, a, a_d)

    Sum = np.sum(a) / np.sqrt(2 * np.pi) / np.float64(n)

    return Sum

platform = cl.get_platforms()[0]
device = platform.get_devices()[0]
ctx = cl.Context([device])

dev_name = device.name
total_memory = device.global_mem_size / 1024.0 / 1024.0 / 1024.0
threads_per_block = device.max_work_group_size
sm_count = device.max_compute_units

print("Found GPU '%s' with %.3f GB of global memory, max %i threads per block, and %i multiprocessors\n" % 
       (dev_name, total_memory, threads_per_block, sm_count))

start = time.time()

Sum = riemannOpenCL(N)

end = time.time()

time_taken = end - start # in seconds

print("Riemann sum pyOpenCL (double precision) for N = %i  : %.17f" % (N, Sum));
print("Total time (measured by CPU)                                : %f s" % time_taken);

Found GPU 'Tesla P100-PCIE-16GB' with 15.899 GB of global memory, max 1024 threads per block, and 56 multiprocessors

OpenCL kernel 'medianTrapezoid' launch with 976563 blocks of 1024 threads

Riemann sum pyOpenCL (double precision) for N = 1000000000  : 0.34134474606853665
Total time (measured by CPU)                                : 4.052927 s


## Bonus examples

### Multiplication table to 100

#### CUDA multiplication table

In [ ]:
%%sh 
cat > mult_table.cu << EOF
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void multTable(int *out, int *a, int *b, int m, int n) {

    int i = blockDim.x * blockIdx.x + threadIdx.x;
    int j = blockDim.y * blockIdx.y + threadIdx.y;
    
    if (i < m || j < n)
    {
        out[i*n+j] = a[i] * b[j];
    }
}

int main(){
    int m = 10;
    int n = 10;

    int *a, *b, *out;
    int *d_a, *d_b, *d_out; 

    // Allocate host memory
    a   = (int*)malloc(sizeof(int) * m);
    b   = (int*)malloc(sizeof(int) * n);
    out = (int*)malloc(sizeof(int) * m * n);

    // Initialize host arrays
    for(int i = 0; i < m; i++) {
        a[i] = i+1;
    }
    for(int i = 0; i < n; i++) {
        b[i] = i+1;
    }

    // Allocate device memory
    cudaMalloc((int**)&d_a, sizeof(int) * m);
    cudaMalloc((int**)&d_b, sizeof(int) * n);
    cudaMalloc((int**)&d_out, sizeof(int) * m * n);

    // Transfer data from host to device memory
    cudaMemcpy(d_a, a, sizeof(int) * m, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(int) * n, cudaMemcpyHostToDevice);

    // Executing kernel
    dim3 dimBlock(5, 2);
    dim3 dimGrid(2, 5);

    multTable<<<dimBlock, dimGrid>>>(d_out, d_a, d_b, m, n);
        
    // Transfer data back to host memory
    cudaMemcpy(out, d_out, sizeof(int) * m * n, cudaMemcpyDeviceToHost);

    for(int i = 0; i < m; i++)
    {
        for(int j = 0; j < n; j++)
        {
            printf("%d * %d = %d\n", a[i], b[j], out[i*n+j]);
        }
        printf("######\n");
    }

    // Deallocate device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);

    // Deallocate host memory
    free(a); 
    free(b); 
    free(out);
}
EOF

In [ ]:
!nvcc -o mult_table mult_table.cu && ./mult_table

1 * 1 = 1
1 * 2 = 2
1 * 3 = 3
1 * 4 = 4
1 * 5 = 5
1 * 6 = 6
1 * 7 = 7
1 * 8 = 8
1 * 9 = 9
1 * 10 = 10
######
2 * 1 = 2
2 * 2 = 4
2 * 3 = 6
2 * 4 = 8
2 * 5 = 10
2 * 6 = 12
2 * 7 = 14
2 * 8 = 16
2 * 9 = 18
2 * 10 = 20
######
3 * 1 = 3
3 * 2 = 6
3 * 3 = 9
3 * 4 = 12
3 * 5 = 15
3 * 6 = 18
3 * 7 = 21
3 * 8 = 24
3 * 9 = 27
3 * 10 = 30
######
4 * 1 = 4
4 * 2 = 8
4 * 3 = 12
4 * 4 = 16
4 * 5 = 20
4 * 6 = 24
4 * 7 = 28
4 * 8 = 32
4 * 9 = 36
4 * 10 = 40
######
5 * 1 = 5
5 * 2 = 10
5 * 3 = 15
5 * 4 = 20
5 * 5 = 25
5 * 6 = 30
5 * 7 = 35
5 * 8 = 40
5 * 9 = 45
5 * 10 = 50
######
6 * 1 = 6
6 * 2 = 12
6 * 3 = 18
6 * 4 = 24
6 * 5 = 30
6 * 6 = 36
6 * 7 = 42
6 * 8 = 48
6 * 9 = 54
6 * 10 = 60
######
7 * 1 = 7
7 * 2 = 14
7 * 3 = 21
7 * 4 = 28
7 * 5 = 35
7 * 6 = 42
7 * 7 = 49
7 * 8 = 56
7 * 9 = 63
7 * 10 = 70
######
8 * 1 = 8
8 * 2 = 16
8 * 3 = 24
8 * 4 = 32
8 * 5 = 40
8 * 6 = 48
8 * 7 = 56
8 * 8 = 64
8 * 9 = 72
8 * 10 = 80
######
9 * 1 = 9
9 * 2 = 18
9 * 3 = 27
9 * 4 = 36
9 * 5 = 45
9 * 6 = 54
9 * 7 = 63
9 

#### OpenCL multiplication table

In [ ]:
%%sh 
cat > mult_table.cl << EOF
__kernel void multTable(__global const int *a,
		        __global const int *b,		
			__global int *out,				
		        int m,
		        int n)						
{														
    int i = get_global_id(0);
    int j = get_global_id(1);

    if (i < m || j < n)
    {
        out[i*n+j] = a[i] * b[j];
    }
}
EOF

In [ ]:
 %%sh 
cat > mult_table_opencl.c << EOF
#include <stdio.h>
#include <stdlib.h>
#include <CL/cl.h>
#include <math.h>

#define MAX_SOURCE_SIZE (0x100000)

int main(void) 
{
    int m = 10;
    int n = 10;

    // Load the kernel source code into the array source_str
    FILE *fp;
    char *source_str;
    size_t source_size;

    fp = fopen("mult_table.cl", "r");
    if (!fp) {
        fprintf(stderr, "Failed to load kernel.\n");
        exit(1);
    }
    source_str = (char*)malloc(MAX_SOURCE_SIZE);
    source_size = fread( source_str, 1, MAX_SOURCE_SIZE, fp);
    fclose( fp );

   // Allocate host memory

    int *a = (int*)malloc(sizeof(int)*m);
    int *b = (int*)malloc(sizeof(int)*n);
    int *out = (int*)malloc(sizeof(int)*m*n);

    // Create the two input arrays
    for(int i = 0; i < m; i++) {
        a[i] = i+1;
    }
    for(int i = 0; i < n; i++) {
        b[i] = i+1;
    }

    // Get platform and device information
    cl_platform_id platform_id = NULL;
    cl_device_id device_id = NULL;   
    cl_uint ret_num_devices;
    cl_uint ret_num_platforms;
    cl_int ret = clGetPlatformIDs(1, &platform_id, &ret_num_platforms);
    ret = clGetDeviceIDs( platform_id, CL_DEVICE_TYPE_ALL, 1, 
            &device_id, &ret_num_devices);
    
    // Create an OpenCL context
    cl_context context = clCreateContext( NULL, 1, &device_id, NULL, NULL, &ret);

    // Create a command queue
    cl_command_queue command_queue = clCreateCommandQueue(context, device_id, 0, &ret);

    // Create memory buffers on the device for each array
    cl_mem a_mem_obj = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, m*sizeof(int), a, &ret);
    cl_mem b_mem_obj = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, n*sizeof(int), b, &ret);
    cl_mem out_mem_obj = clCreateBuffer(context, CL_MEM_WRITE_ONLY, m*n*sizeof(int), NULL, &ret);
  
    // Create a program from the kernel source
    cl_program program = clCreateProgramWithSource(context, 1, (const char **)&source_str, NULL, &ret);						
    
    // Build the program
    ret = clBuildProgram(program, 1, &device_id, NULL, NULL, NULL);
		
    // Create the OpenCL kernel
    cl_kernel kernel = clCreateKernel(program, "multTable", &ret);
 
    // Set the arguments of the kernel
    clSetKernelArg(kernel, 0, sizeof(cl_mem), (void *)&a_mem_obj);
    clSetKernelArg(kernel, 1, sizeof(cl_mem), (void *)&b_mem_obj);
    clSetKernelArg(kernel, 2, sizeof(cl_mem), (void *)&out_mem_obj);
    clSetKernelArg(kernel, 3, sizeof(cl_int), (void *)&m);
    clSetKernelArg(kernel, 4, sizeof(cl_int), (void *)&n);


    // Execute the OpenCL kernel
    size_t local_item_size[2] = {5, 2};
    size_t global_item_size[2] = {m, n};

    ret = clEnqueueNDRangeKernel(command_queue, kernel, 2, NULL, global_item_size, local_item_size, 0, NULL, NULL);

    // Read the memory buffer out on the device to the local variable out
    ret = clEnqueueReadBuffer(command_queue, out_mem_obj, CL_TRUE, 0, m*n*sizeof(int), out, 0, NULL, NULL);				
			

    // Display the results to the screen
    for(int i = 0; i < m; i++)
    {
        for(int j = 0; j < n; j++)
        {
            printf("%d * %d = %d\n", a[i], b[j], out[i*n+j]);
        }
        printf("######\n");
    }
    
    // Clean up
    ret = clFlush(command_queue);
    ret = clFinish(command_queue);
    ret = clReleaseKernel(kernel);
    ret = clReleaseProgram(program);
    ret = clReleaseMemObject(a_mem_obj);
    ret = clReleaseMemObject(b_mem_obj);
    ret = clReleaseMemObject(out_mem_obj);
    ret = clReleaseCommandQueue(command_queue);
    ret = clReleaseContext(context);
	
    free(a);
    free(b);
    free(out);

    return 0;
}
EOF

In [ ]:
!nvcc -o mult_table_opencl mult_table_opencl.c -lOpenCL && ./mult_table_opencl

1 * 1 = 1
1 * 2 = 2
1 * 3 = 3
1 * 4 = 4
1 * 5 = 5
1 * 6 = 6
1 * 7 = 7
1 * 8 = 8
1 * 9 = 9
1 * 10 = 10
######
2 * 1 = 2
2 * 2 = 4
2 * 3 = 6
2 * 4 = 8
2 * 5 = 10
2 * 6 = 12
2 * 7 = 14
2 * 8 = 16
2 * 9 = 18
2 * 10 = 20
######
3 * 1 = 3
3 * 2 = 6
3 * 3 = 9
3 * 4 = 12
3 * 5 = 15
3 * 6 = 18
3 * 7 = 21
3 * 8 = 24
3 * 9 = 27
3 * 10 = 30
######
4 * 1 = 4
4 * 2 = 8
4 * 3 = 12
4 * 4 = 16
4 * 5 = 20
4 * 6 = 24
4 * 7 = 28
4 * 8 = 32
4 * 9 = 36
4 * 10 = 40
######
5 * 1 = 5
5 * 2 = 10
5 * 3 = 15
5 * 4 = 20
5 * 5 = 25
5 * 6 = 30
5 * 7 = 35
5 * 8 = 40
5 * 9 = 45
5 * 10 = 50
######
6 * 1 = 6
6 * 2 = 12
6 * 3 = 18
6 * 4 = 24
6 * 5 = 30
6 * 6 = 36
6 * 7 = 42
6 * 8 = 48
6 * 9 = 54
6 * 10 = 60
######
7 * 1 = 7
7 * 2 = 14
7 * 3 = 21
7 * 4 = 28
7 * 5 = 35
7 * 6 = 42
7 * 7 = 49
7 * 8 = 56
7 * 9 = 63
7 * 10 = 70
######
8 * 1 = 8
8 * 2 = 16
8 * 3 = 24
8 * 4 = 32
8 * 5 = 40
8 * 6 = 48
8 * 7 = 56
8 * 8 = 64
8 * 9 = 72
8 * 10 = 80
######
9 * 1 = 9
9 * 2 = 18
9 * 3 = 27
9 * 4 = 36
9 * 5 = 45
9 * 6 = 54
9 * 7 = 63
9 